In [28]:
import yaml
import snowflake.connector
from databricks import sql
import pandas as pd
from IPython.display import display

# 🔹 Load Configuration from YAML
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

sf_config = config["snowflake"]
db_config = config["databricks"]
validation_config = config["validation"]["unique_columns"]

# 🔹 Establish Snowflake Connection
conn_sf = snowflake.connector.connect(
    user=sf_config["user"],
    account=sf_config["account"],
    authenticator=sf_config["authenticator"],
    database=sf_config["database"],
    schema=sf_config["schema"]
)
cur_sf = conn_sf.cursor()

# 🔹 Establish Databricks Connection
conn_db = sql.connect(
    server_hostname=db_config["server_hostname"],
    http_path=db_config["http_path"],
    access_token=db_config["access_token"]
)
cur_db = conn_db.cursor()

# 🔹 Store Validation Results
validation_results = []

# 🔹 Loop through each NBCU Table
for table in validation_config.keys():
    print(f"🔍 Validating Table: {table}")

    # 🔹 Fetch Row Count from Snowflake
    try:
        query_sf = f"SELECT COUNT(*) FROM {table}"
        cur_sf.execute(query_sf)
        snowflake_row_count = cur_sf.fetchone()[0]
    except Exception as e:
        print(f"❌ Error fetching row count from Snowflake for {table}: {e}")
        snowflake_row_count = None

    # 🔹 Fetch Row Count from Databricks
    try:
        query_db = f"SELECT COUNT(*) FROM {db_config['database']}.{table.lower()}"
        cur_db.execute(query_db)
        databricks_row_count = cur_db.fetchone()[0]
    except Exception as e:
        print(f"❌ Error fetching row count from Databricks for {table}: {e}")
        databricks_row_count = None

    # 🔹 Fetch Unique Values from Snowflake
    unique_column = validation_config[table]
    try:
        query_sf_unique = f"SELECT COUNT(DISTINCT {unique_column}) FROM {table}"
        cur_sf.execute(query_sf_unique)
        snowflake_unique_values = cur_sf.fetchone()[0]
    except Exception as e:
        print(f"❌ Error fetching unique values from Snowflake for {table}: {e}")
        snowflake_unique_values = None

    # 🔹 Fetch Unique Values from Databricks
    try:
        query_db_unique = f"SELECT COUNT(DISTINCT {unique_column}) FROM {db_config['database']}.{table.lower()}"
        cur_db.execute(query_db_unique)
        databricks_unique_values = cur_db.fetchone()[0]
    except Exception as e:
        print(f"❌ Error fetching unique values from Databricks for {table}: {e}")
        databricks_unique_values = None

    # 🔹 Fetch Column Structure from Snowflake
    try:
        cur_sf.execute(f"DESCRIBE TABLE {table}")
        snowflake_columns = [row[0] for row in cur_sf.fetchall()]
    except Exception as e:
        print(f"❌ Error fetching schema from Snowflake for {table}: {e}")
        snowflake_columns = []

    # 🔹 Fetch Column Structure from Databricks
    try:
        cur_db.execute(f"DESCRIBE TABLE {db_config['database']}.{table.lower()}")
        databricks_columns = [row[0] for row in cur_db.fetchall()]
    except Exception as e:
        print(f"❌ Error fetching schema from Databricks for {table}: {e}")
        databricks_columns = []

    # 🔹 Store Results
    validation_results.append({
        "table_name": table,
        "snowflake_rows": snowflake_row_count,
        "databricks_rows": databricks_row_count,
        "row_count_match": snowflake_row_count == databricks_row_count,
        "snowflake_unique_values": snowflake_unique_values,
        "databricks_unique_values": databricks_unique_values,
        "unique_values_match": snowflake_unique_values == databricks_unique_values,
        "snowflake_columns": snowflake_columns,
        "databricks_columns": databricks_columns,
        "schema_match": set(snowflake_columns) == set(databricks_columns)
    })

# 🔹 Convert to DataFrame
validation_df = pd.DataFrame(validation_results)

# 🔹 Display Results
print("✅ Validation Summary:")
display(validation_df)

# 🔹 Close Connections
cur_sf.close()
conn_sf.close()
cur_db.close()
conn_db.close()
print("✅ Validation Completed & Connections Closed.")


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/b1aae949-a5ef-4815-b7af-f7c4aa546b28/saml2?SAMLRequest=nZLBctowEIZfxaOebcsGg9EAGRdKS5NSik1n2puw10TFlohWxuHtK0yYSQ%2FJoTeN9O3qk%2F4d3z3XlXMCjULJCQk8ShyQuSqE3E%2FINlu4MXHQcFnwSkmYkDMguZuOkdfVkSWNeZQbeGoAjWMbSWTdwYQ0WjLFUSCTvAZkJmdp8u2BhR5lR62MylVFXpW8X8ERQRtreCspUFi9R2OOzPfbtvXanqf03g8ppT4d%2BZa6IB9u%2FLN90xt84NP%2BhbeExdcvbh%2BFvH7Be1q7K4TsS5at3fX3NCNOclOdKYlNDToFfRI5bDcPVwG0BrPw8zrZZKtPm9Rr0AWOxg08lKotK36AXNXHxtjOnl35JRR%2BpfbCPn45n5DjQRRwH38tn7iIT6v0EK%2BS7fnPj%2FCXPg3Op4y2i8MAeSqKaLvSNCfOz1u64SXdJWIDS3nJ1NgtGkYu7bl0mNEeCyiLIm8QxL%2BJM7eZCslNV3kT7zy8WuRaoSqNkpWQ0FnuAs5h1B%2B5PILS7cdB5O6GvHTLYd7nPOoPdmHsX5IOyXV6WCeip%2F%2FzJ2P%2FdYeXYVzZfJbztapEfnYWStfcvB1f4AXdjijcskMZ1FxUSVFoQLQxVpVqZxq4sTNvdAPEn15v%2FXfqp3

,table_name,snowflake_rows,databricks_rows,row_count_match,snowflake_unique_values,databricks_unique_values,unique_values_match,snowflake_columns,databricks_columns,schema_match
0,NBCU_CUSTOMERS,20,20,True,20,20,True,"[CUSTOMER_ID, CUSTOMER_NAME, INDUSTRY, CONTACT...","[customer_id, customer_name, industry, contact...",False
1,NBCU_DEPARTMENTS,5,5,True,5,5,True,"[DEPARTMENT_ID, DEPARTMENT_NAME, LOCATION]","[department_id, department_name, location]",False
2,NBCU_EMPLOYEES,30,30,True,30,30,True,"[EMPLOYEE_ID, FIRST_NAME, LAST_NAME, DEPARTMEN...","[employee_id, first_name, last_name, departmen...",False
3,NBCU_PRODUCTS,15,15,True,15,15,True,"[PRODUCT_ID, PRODUCT_NAME, CATEGORY, RELEASE_D...","[product_id, product_name, category, release_d...",False
4,NBCU_PROJECTS,10,10,True,10,10,True,"[PROJECT_ID, PROJECT_NAME, DEPARTMENT_ID, STAR...","[project_id, project_name, department_id, star...",False
5,NBCU_SALES,100,100,True,100,100,True,"[SALE_ID, PRODUCT_ID, CUSTOMER_ID, SALE_DATE, ...","[sale_id, product_id, customer_id, sale_date, ...",False
6,NBCU_SUPPLIERS,50,50,True,50,50,True,"[SUPPLIER_ID, SUPPLIER_NAME, MATERIAL_SUPPLIED...","[supplier_id, supplier_name, material_supplied...",False
7,NBCU_SUPPLY_CHAIN,100,100,True,100,100,True,"[SUPPLY_CHAIN_ID, SUPPLIER_ID, PRODUCT_ID, SUP...","[supply_chain_id, supplier_id, product_id, sup...",False


✅ Validation Completed & Connections Closed.
